In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install transformers --upgrade

In [ ]:
!pip install peft

In [ ]:
!pip install datasets --upgrade

In [ ]:
import pandas as pd
from datasets import load_dataset, load_metric, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import get_peft_config, LoraConfig, TaskType, get_peft_model

# Create DataFrames for training and evaluation data
"""
train_texts = [
    "This is a positive example.",
    "This is another positive example.",
    "This is a negative example.",
    # Add more training examples here
]

train_labels = [1, 1, 0]

eval_texts = [
    "This is a positive evaluation example.",
    "This is a negative evaluation example.",
    # Add more evaluation examples here
]

eval_labels = [1, 0]

train_data = pd.DataFrame({"text": train_texts, "label": train_labels})
eval_data = pd.DataFrame({"text": eval_texts, "label": eval_labels})
"""

In [ ]:
import string 
import re
translator = str.maketrans('', '', string.punctuation) 

In [ ]:
def calc_target(row):
    orig_text = row['orig_text'].translate(translator).split()
    text = row['text'].translate(translator).split()
    return 1 if orig_text == text  else 0

In [ ]:
train_data = pd.read_csv('/kaggle/input/michael-ivrit-dataset/train_data.csv', index_col=None).dropna()
eval_data = pd.read_csv('/kaggle/input/michael-ivrit-dataset/test_data.csv', index_col=None).dropna()
train_data['label'] = train_data.apply(calc_target, axis=1)
eval_data['label'] = train_data.apply(calc_target, axis=1)

train_data = train_data[['orig_text', 'label']].rename(columns={'orig_text': 'text'})
eval_data = eval_data[['orig_text', 'label']].rename(columns={'orig_text': 'text'})
# Create a dataset object
train_dataset = Dataset.from_pandas(train_data)
eval_dataset = Dataset.from_pandas(train_data)

In [ ]:
# Load the pre-trained tokenizer and model
model_name = "xlm-roberta-large"
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Binary classification

# Tokenize the texts
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

# Configure the LoRA parameters
# peft_config = LoraConfig(model, TaskType.SEQ_CLS, LoraConfig(r=16, lora_alpha=32))
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, inference_mode=False, r=16, lora_alpha=32, lora_dropout=0.1
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
# Initialize the Trainer
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=1e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=50,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to='tensorboard'
)

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Evaluate the model
eval_result = trainer.evaluate()

print(f"Evaluation result: {eval_result}")

In [ ]:
model.save_pretrained("output_dir")